In [1]:
import os
from pytube import YouTube
import subprocess

In [32]:
link = 'https://youtu.be/Qp0j29sl-0E'

name = 'Анатомия для художников 3D скульптинг для начинающих'

path = "d:\youtube"

In [21]:
yt = YouTube(link)

# download video
r_video = yt.streams.filter(resolution="1080p").first().download(path, "_video.mp4")
print(r_video)
# download audio
r_audio = yt.streams.filter(only_audio=True).first().download(path, "_audio.mp4")
print(r_audio)


d:\youtube\_video.mp4
d:\youtube\_audio.mp4


In [33]:
#name = name.replace(' ','_')
# Создание исполняемого файла
f = open(os.path.join(path, '_ffmpeg.bat') , 'w', encoding='cp866')
f.write('ffmpeg -i ')
f.write(os.path.join(path, "_video.mp4"))
f.write(' -i ')
f.write(os.path.join(path, "_audio.mp4"))
f.write(' -c copy ')
f.write('\"')
f.write(os.path.join(path, name + ".mp4"))
f.write('\"')
#f.write('\n')
#f.write('pause')
f.close()

In [30]:
# Объединение видео с аудио
subprocess.Popen(os.path.join(path, '_ffmpeg.bat'))

<Popen: returncode: None args: 'd:\\youtube\\_ffmpeg.bat'>

In [20]:
# Удаляем файлы
os.remove(os.path.join(path, "_video.mp4"))
os.remove(os.path.join(path, "_audio.mp4"))
os.remove(os.path.join(path, '_ffmpeg.bat'))